In [7]:
import albumentations as A
from typing import Callable
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os

In [8]:
# read images from folder
def read_images_from_folder(folder_path: str) -> list:   
    images = []
    for file in os.listdir(folder_path):
        if file.endswith(".png"):
            image = cv.imread(os.path.join(folder_path, file))
            img_arr = np.array(image)
            images.append(img_arr)
    return images

In [9]:
def resize_image(image, width, height):
    resized = cv.resize(image, (width, height), interpolation = cv.INTER_AREA)
    return resized

In [10]:
trans = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.OneOf([
        A.MotionBlur(p=0.2), # use motion blur 
        A.MedianBlur(blur_limit=3, p=0.1), # use median blur
        A.Blur(blur_limit=3, p=0.1), # use blur
    ], p=0.2), 
    A.OneOf([
        A.GaussNoise(), # add gauss noise
    ], p=0.2),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2), # shift scale rotate
    A.RandomBrightnessContrast(p=0.2), # random brightness contrast
])

In [11]:
def image_augmentation(image, transform):
    """
    Apply augmentation to the image.
    :param image: image to be augmented
    :param transform: augmentation to be applied
    :return: augmented image
    """
    return transform(image=image)['image']

In [12]:
images_coins = read_images_from_folder("coins")
augmented_images = []

for index, image in enumerate(images_coins):
    resized_image = resize_image(image, 600, 600)
    augmented_image = image_augmentation(image = resized_image, transform = trans)
    augmented_images.append(augmented_image)
    cv.imwrite(f"coins/{index}_augmented.png", augmented_image)
    